# Table of contents

* [Preprocessing](#Preprocessing)
    + [Missing Value Handling](#Missing-Value-Handling)
    + [Remove duplicates](#Remove-duplicates)
    + [Outlier Handling](#Outlier-Handling)
* [Analyze](#Analyze)
    + [1. 서울시 자치구별 학원, 교습소 수](#1.-서울시-자치구별-학원,-교습소-수)
    + [2. 서울시 자치구별 학원, 교습소 정원 수(온라인, 원격 학원/교습소 포함)](#2.-서울시-자치구별-학원,-교습소-정원-수(온라인,-원격-학원/교습소-포함))
    + [3. 서울시 학군별 학원/교습소 & 학교 수](#3.-서울시-학군별-학원/교습소-&-학교-수)
    + [4. 서울시 학군별 학원/교습소 학생 수 & 학교 학생 수](#4.서울시-학군별-학교-학생-수-&-학원/교습소-학생-수)
* [Visualize](#Visualize)
    + [1. 서울시 자치구별 학원, 교습소 수 시각화](#1.-서울시-자치구별-학원,-교습소-수-시각화)
    + [2. 서울시 학군별 학원/교습소 & 학교 수 시각화](#2.-서울시-학군별-학원/교습소-&-학교-수-시각화)
    + [3. 학군별 학원/교습소 학생 수(온라인, 원격 제외) & 학교 학생 수 시각화](#3.-학군별-학원/교습소-학생-수(온라인,-원격-제외)-&-학교-학생-수-시각화)

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium

%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

ModuleNotFoundError: No module named 'folium'

### Import Data

In [4]:
# Main Data
academy = pd.read_csv('./data/서울특별시 학원 교습소정보.csv')
school = pd.read_csv('./data/서울특별시 학교 기본정보.csv')
school_cnt = pd.read_excel('./data/2021 서울시 학급당 학생수 통계.xlsx') # 외부데이터
# 전처리 한 데이터를 시트별로 통합 / 그 중 '학원교습소정보'와 '학교기본정보' 시트 사용
df = pd.read_excel('./data/helloworld_data_set.xlsx', sheet_name=['학원교습소정보', '학교기본정보'] )
locals().update(df)

### Preprocessing
전처리 수행(결측값 처리, 중복값 처리, 이상치 처리 등)

In [5]:
# Dtype 확인
academy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25606 entries, 0 to 25605
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   행정구역명       25537 non-null  object 
 1   학원/교습소      25606 non-null  object 
 2   학원명         25606 non-null  object 
 3   도로명주소       25594 non-null  object 
 4   도로명상세주소     25509 non-null  object 
 5   분야명         25606 non-null  object 
 6   교습계열명       25159 non-null  object 
 7   교습과정명       25161 non-null  object 
 8   정원합계        25606 non-null  int64  
 9   일시수용능력인원합계  25528 non-null  float64
 10  인당수강료내용     4608 non-null   object 
 11  기숙사학원여부     23677 non-null  object 
 12  등록일자        25606 non-null  int64  
 13  휴원시작일자      353 non-null    float64
 14  휴원종료일자      353 non-null    float64
 15  개설일자        25606 non-null  int64  
dtypes: float64(3), int64(3), object(10)
memory usage: 3.1+ MB


In [6]:
# 열별 결측치 개수 확인
academy.isnull().sum()

행정구역명            69
학원/교습소            0
학원명               0
도로명주소            12
도로명상세주소          97
분야명               0
교습계열명           447
교습과정명           445
정원합계              0
일시수용능력인원합계       78
인당수강료내용       20998
기숙사학원여부        1929
등록일자              0
휴원시작일자        25253
휴원종료일자        25253
개설일자              0
dtype: int64

### Missing Value Handling

In [7]:
# 도로명상세주소 + 도로명주소 + 교습계열명 + 교습과정명 + 일시수용능력인원합계 결측치 포함 행 제거
academy = academy.dropna(subset=['도로명상세주소', '교습계열명', '일시수용능력인원합계']) 

In [8]:
academy['행정구역명'] = academy['행정구역명'].fillna(academy['도로명주소'].str.split(' ').str[1])
academy['인당수강료내용'] = academy['인당수강료내용'].fillna('알수없음')
academy['기숙사학원여부'] = academy['기숙사학원여부'].fillna('알수없음')
academy['휴원시작일자'] = academy['휴원시작일자'].fillna(0)
academy['휴원종료일자'] = academy['휴원종료일자'].fillna(0)

In [9]:
# '서울특별시마포구'같이 띄어쓰기가 안 되어 있는 데이터가 있어 '서울특별시'를 제거하고 앞의 공백을 제거
academy['도로명주소'] = academy['도로명주소'].str.replace('서울특별시', '')
academy['도로명주소'].str.lstrip().tolist()

['동대문구 왕산로33길 32',
 '동대문구 고산자로36길 3',
 '동대문구 이문로 38',
 '동대문구 왕산로 31',
 '중랑구 동일로151길 30',
 '중랑구 공릉로 58',
 '중랑구 동일로101길 52',
 '동대문구 한천로 124',
 '동대문구 사가정로 190',
 '동대문구 천호대로16길 12',
 '중랑구 동일로129길 54',
 '동대문구 장한로30길 31',
 '동대문구 답십리로72길 118',
 '중랑구 겸재로3길 22',
 '동대문구 이문로1길 16-3',
 '동대문구 이문로1길 16-3',
 '동대문구 천호대로77길 89',
 '동대문구 황물로 42',
 '동대문구 답십리로23길 59',
 '중랑구 면목로48길 49',
 '동대문구 한천로 165',
 '중랑구 용마산로 389',
 '동대문구 하정로 36',
 '동대문구 홍릉로15길 43',
 '중랑구 면목로 306',
 '동대문구 이문로 65',
 '중랑구 동일로140길 16',
 '동대문구 휘경로 39',
 '중랑구 동일로129길 22',
 '중랑구 용마산로 478',
 '중랑구 면목로27길 6',
 '중랑구 용마산로 436',
 '동대문구 답십리로 133',
 '중랑구 용마산로116길 22',
 '동대문구 답십리로66길 99',
 '중랑구 봉화산로56길 153',
 '중랑구 면목로27길 6',
 '중랑구 봉우재로 211',
 '중랑구 용마산로94길 47',
 '중랑구 면목로48길 65',
 '동대문구 제기로38길 50',
 '중랑구 봉우재로5길 4',
 '동대문구 한천로 481',
 '동대문구 천호대로77길 94',
 '동대문구 왕산로 249-1',
 '동대문구 사가정로 93',
 '중랑구 신내로7가길 19',
 '동대문구 답십리로72길 112',
 '중랑구 중랑역로 79',
 '동대문구 제기로 131',
 '동대문구 제기로 131',
 '동대문구 한천로 246',
 '중랑구 봉화산로53길19',
 '중랑구 신내로 71',
 '중랑구 면목로55길 28',
 '중랑구 동일로14

In [10]:
# 결측치 처리 확인
academy.isnull().sum()

행정구역명         0
학원/교습소        0
학원명           0
도로명주소         0
도로명상세주소       0
분야명           0
교습계열명         0
교습과정명         0
정원합계          0
일시수용능력인원합계    0
인당수강료내용       0
기숙사학원여부       0
등록일자          0
휴원시작일자        0
휴원종료일자        0
개설일자          0
dtype: int64

### Remove duplicates

In [11]:
academy.duplicated().sum() # 중복값 1개 확인

1

In [12]:
academy.shape # 중복값 제거 전 행,열 개수

(24984, 16)

In [13]:
academy.drop_duplicates(inplace=True) # 중복값 제거

In [14]:
academy.shape # 중복값 제거 후 행, 열 개수

(24983, 16)

### Outlier Handling

In [15]:
# 이상치 처리 전
academy.describe()

,정원합계,일시수용능력인원합계,등록일자,휴원시작일자,휴원종료일자,개설일자
count,2.498300e+04,2.498300e+04,2.498300e+04,2.498300e+04,2.498300e+04,2.498300e+04
mean,4.328286e+03,4.005378e+05,2.012343e+07,2.802403e+05,2.842947e+05,2.012455e+07
std,3.933769e+05,6.326707e+07,1.395984e+05,2.361351e+06,2.444597e+06,8.092310e+04
min,0.000000e+00,0.000000e+00,2.210707e+06,0.000000e+00,0.000000e+00,1.953052e+07
25%,1.900000e+01,6.000000e+00,2.008121e+07,0.000000e+00,0.000000e+00,2.009011e+07
50%,4.700000e+01,4.200000e+01,2.014122e+07,0.000000e+00,0.000000e+00,2.014122e+07
75%,1.250000e+02,7.800000e+01,2.019010e+07,0.000000e+00,0.000000e+00,2.019011e+07
max,5.771500e+07,1.000000e+10,2.106091e+07,2.022012e+07,1.000000e+08,2.022032e+07


In [16]:
academy['휴원종료일자'].unique() # 이상치 99999999

array([       0., 20110828., 20210115., 20101028., 20180105., 20150515.,
       20140505., 20140805., 20130406., 20170801., 20141231., 20201025.,
       20160103., 20180713., 20190203., 20181207., 20111223., 20180613.,
       20170123., 20111231., 20200430., 20210131., 20200419., 20200415.,
       20200322., 20200426., 20200405., 20200313., 20201231., 20170430.,
       20200417., 20181118., 20141206., 20161008., 20111215., 20110316.,
       20200731., 20210516., 20170315., 20200403., 20161231., 20140916.,
       20200228., 20150430., 20121231., 20160429., 20210830., 20211130.,
       20141116., 20130912., 20110424., 20141012., 99999999., 20140803.,
       20110831., 20200327., 20150228., 20180707., 20210228., 20120415.,
       20160831., 20120513., 20180921., 20110925., 20211004., 20140416.,
       20150821., 20210205., 20151212., 20200910., 20200420., 20200505.,
       20141121., 20120812., 20180831., 20210811., 20200830., 20170501.,
       20180720., 20150823., 20120701., 20131231., 

In [17]:
academy.query('휴원종료일자 == 99999999') # 이상치 포함 행 확인 / 휴원시작일자 0이므로 휴원종료일자도 0으로 대체

,행정구역명,학원/교습소,학원명,도로명주소,도로명상세주소,분야명,교습계열명,교습과정명,정원합계,일시수용능력인원합계,인당수강료내용,기숙사학원여부,등록일자,휴원시작일자,휴원종료일자,개설일자
2898,양천구,교습소,강해수학교습소,양천구 목동서로 77,현대월드타워 505호 (목동),입시.검정 및 보습,보통교과,보습,4,2.0,수학:443160/ 수학:531800,알수없음,20070328,0.0,99999999.0,20070328


In [18]:
# 휴원종료일자 이상치 99999999를 0으로 대체 (해당 행 휴원시작일자 = 0)
academy['휴원종료일자'] = academy['휴원종료일자'].replace(99999999, 0) 

# 등록일자 오타 수정
academy['등록일자'] = academy['등록일자'].replace(21060906, 20160906) 
academy['등록일자'] = academy['등록일자'].replace(2210707, 20210707)

In [19]:
academy.select_dtypes('object').describe() # 이상치 없음

,행정구역명,학원/교습소,학원명,도로명주소,도로명상세주소,분야명,교습계열명,교습과정명,인당수강료내용,기숙사학원여부
count,24983,24983,24983,24983,24983,24983,24983,24983,24983,24983
unique,25,2,22982,13019,19079,11,20,109,4403,3
top,강남구,학원,오하운폴댄스학원,양천구 목동서로 77,/ 2층 (대치동),입시.검정 및 보습,보통교과,보습,알수없음,N
freq,3426,14642,9,142,87,13536,13426,11086,20414,23051


In [20]:
# 이?미용 -> 이·미용, 보습?논술 -> 보습·논술 / 데이터 출처 찾아서 ?를 ·로 수정
academy['교습과정명'] = academy['교습과정명'].str.replace('?', '·') 
academy['교습계열명'] = academy['교습계열명'].str.replace('?', '·') 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6236\2478664403.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  academy['교습과정명'] = academy['교습과정명'].str.replace('?', '·')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6236\2478664403.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  academy['교습계열명'] = academy['교습계열명'].str.replace('?', '·')


In [21]:
# 일시수용능력인원합계 9999명 이상을 이상치로 보고 0으로 처리 / 9999명 이상 학원/교습소들은 온라인, 원격으로 수업or병행 진행
c1 = academy['일시수용능력인원합계'] < 9999
academy['일시수용능력인원합계'] = academy['일시수용능력인원합계'].where(c1, 0)

# 정원합계 50000명 이상을 이상치로 보고 해당 행 삭제(50000명 이상인 학원/교습소 = 원격, 온라인 학원)
c2 = academy['정원합계'] < 50000
academy['정원합계'] = academy['정원합계'].where(c2, np.nan)
academy = academy.dropna()

# 이상치 처리 후
academy.describe()

,정원합계,일시수용능력인원합계,등록일자,휴원시작일자,휴원종료일자,개설일자
count,24939.000000,24939.000000,2.493900e+04,2.493900e+04,2.493900e+04,2.493900e+04
mean,247.579414,56.769437,2.012406e+07,2.807347e+05,2.807865e+05,2.012450e+07
std,1390.632880,87.895622,8.131842e+04,2.363404e+06,2.363838e+06,8.094829e+04
min,0.000000,0.000000,1.953052e+07,0.000000e+00,0.000000e+00,1.953052e+07
25%,19.000000,6.000000,2.008121e+07,0.000000e+00,0.000000e+00,2.009011e+07
50%,46.000000,42.000000,2.014122e+07,0.000000e+00,0.000000e+00,2.014122e+07
75%,124.000000,77.000000,2.019010e+07,0.000000e+00,0.000000e+00,2.019011e+07
max,49672.000000,4000.000000,2.022032e+07,2.022012e+07,2.022123e+07,2.022032e+07


In [22]:
# 등록일자, 개설일자 Dtype object에서 datetime으로 변경
academy['등록일자'] = pd.to_datetime(academy['등록일자'], format='%Y%m%d')
academy['개설일자'] = pd.to_datetime(academy['개설일자'], format='%Y%m%d')

In [23]:
# Dtype 변경 후
academy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24939 entries, 0 to 25605
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   행정구역명       24939 non-null  object        
 1   학원/교습소      24939 non-null  object        
 2   학원명         24939 non-null  object        
 3   도로명주소       24939 non-null  object        
 4   도로명상세주소     24939 non-null  object        
 5   분야명         24939 non-null  object        
 6   교습계열명       24939 non-null  object        
 7   교습과정명       24939 non-null  object        
 8   정원합계        24939 non-null  float64       
 9   일시수용능력인원합계  24939 non-null  float64       
 10  인당수강료내용     24939 non-null  object        
 11  기숙사학원여부     24939 non-null  object        
 12  등록일자        24939 non-null  datetime64[ns]
 13  휴원시작일자      24939 non-null  float64       
 14  휴원종료일자      24939 non-null  float64       
 15  개설일자        24939 non-null  datetime64[ns]
dtypes: datetime64[ns](2), 

In [24]:
# 전처리와 이상치 처리 후 인덱스 번호 재설정
academy.reset_index(drop=True, inplace=True)

### Analyze
데이터 분석(상관 관계, 추세, 변동 등)

#### 1. 서울시 자치구별 학원, 교습소 수

In [25]:
# '학원/교습소' column에서 학원과 교습소로 데이터 구분
edu_institute_01 = academy[academy['학원/교습소'] == '학원'] # = academy.groupby('학원/교습소').get_group('학원')
edu_institute_02 = academy[academy['학원/교습소'] != '학원'] # = cademy.groupby('학원/교습소').get_group('교습소')

# column 이름 재설정
edu_institute_01 = edu_institute_01.rename(columns={'학원/교습소':'학원'})
edu_institute_01 = edu_institute_01.rename(columns={'정원합계':'학원 정원합계'})
edu_institute_02 = edu_institute_02.rename(columns={'학원/교습소':'교습소'})
edu_institute_02 = edu_institute_02.rename(columns={'정원합계':'교습소 정원합계'})

In [26]:
academy_df1 = edu_institute_01.groupby('행정구역명')[['학원']].count() # 자치구별 학원 수
academy_df2 = edu_institute_02.groupby('행정구역명')[['교습소']].count() # 자치구별 교습소 수
df1 = pd.concat([academy_df1,academy_df2],axis=1).sort_values(by='학원', ascending=False) ; df1
# 상위 4개 학원 교습소 밀집 자치구 = 강남구, 서초구, 송파구, 양천구

,학원,교습소
행정구역명,,
강남구,2460,957
서초구,1168,673
송파구,1144,760
양천구,1084,1040
노원구,760,673
강서구,733,574
강동구,710,539
마포구,688,415
은평구,544,520


#### 2. 서울시 자치구별 학원, 교습소 정원 수(온라인, 원격 학원/교습소 포함)

- 학원 학생 수 1위인 강남구는 학원 수도 1위이다. 그러나 두 번째로 학원 학생 수가 많은 동작구는 학원 수 순위 10위이다.

In [ ]:
# 자치구별 학원과 교습소 학생 수 합계
academy_df3 = edu_institute_01.groupby('행정구역명')[['학원 정원합계']].sum()
academy_df4 = edu_institute_02.groupby('행정구역명')[['교습소 정원합계']].sum()
# 학원 정원합계를 기준으로 내림차순 정렬
df2 = pd.concat([academy_df3,academy_df4],axis=1).sort_values(by='학원 정원합계', ascending=False) ; df2

#### 3. 서울시 학군별 학원/교습소 & 학교 수

In [ ]:
# column = 자치구, 학원/교습소, 학원학생수 인덱싱하여 edu_df 생성
col01 = {'자치구': 학원교습소정보['자치구'],
       '학원/교습소': 학원교습소정보['학원/교습소'],
        '학원학생수': 학원교습소정보['정원합계']}
edu_df01 = pd.DataFrame(col01) ; edu_df01

In [ ]:
# column = 자치구, 학교종류명 인덱싱하여 school_df 생성
col02 = {'자치구': 학교기본정보['행정구역명'],
       '학교종류명': 학교기본정보['학교종류명']}
school_df = pd.DataFrame(col02) ; school_df

In [ ]:
# edu_df 자치구별 해당하는 학군열 추가
school_district_lst = ['1학군', '2학군', '3학군', '4학군','5학군', '6학군', '7학군', '8학군', '9학군', '10학군', '11학군']
gu_lst = [['동대문구', '중랑구'], ['마포구', '서대문구', '은평구'], ['구로구', '금천구', '영등포구'], ['노원구', '도봉구'], 
                ['용산구', '종로구', '중구'], ['강동구', '송파구'], ['강서구', '양천구'], ['강남구', '서초구'], ['관악구', '동작구'], 
                ['광진구', '성동구'], '강북구', '성북구']
col = edu_df01['자치구']

for i in range(11):
    globals()['edu_district_'+str(i+1)] = edu_df01.query("@col in @gu_lst[@i]")
    

edu_data_lst = [edu_district_1, edu_district_2, edu_district_3, edu_district_4, edu_district_5, edu_district_6, 
            edu_district_7, edu_district_8, edu_district_9, edu_district_10, edu_district_11]


for k, v in zip(edu_data_lst, school_district_lst):
    k.insert(0, '학군명', np.full([len(k['자치구'])], v))
    
    
edu_district01 = pd.concat([edu_district_1, edu_district_2, edu_district_3,  edu_district_4,  edu_district_5, 
                          edu_district_6, edu_district_7, edu_district_8, edu_district_9, edu_district_10, edu_district_11])
edu_district01

In [ ]:
# school_df 자치구별 해당하는 학군열 추가
col = school_df['자치구']

for i in range(11):
    globals()['school_district_'+str(i+1)] = school_df.query("@col in @gu_lst[@i]")


school_data_lst = [school_district_1, school_district_2, school_district_3, school_district_4, school_district_5, school_district_6, 
            school_district_7, school_district_8, school_district_9, school_district_10, school_district_11]


for k, v in zip(school_data_lst, school_district_lst):
    k.insert(0, '학군명', np.full([len(k['자치구'])], v))
    
    
school_district = pd.concat([school_district_1, school_district_2, school_district_3, school_district_4, school_district_5, 
                             school_district_6, school_district_7, school_district_8, school_district_9, school_district_10, 
                             school_district_11])
school_district

In [ ]:
# 학군별 학원/교습소 수 & 학원학생수
my_dict = {'학원/교습소':'count',
           '학원학생수': 'sum'}
edu_district01 = edu_district01.groupby(['학군명']).agg(my_dict) ; edu_district01

In [ ]:
# 학군별 학교 수
school_district = school_district.groupby('학군명')[['학교종류명']].count() ; school_district

In [ ]:
# column 이름 변경(학교종류명 > 학교수)
school_district.rename(columns={'학교종류명':'학교수'}, inplace=True) 

In [ ]:
# 학군별 학원/교습소 수와 학교수 merge
edu_school = edu_district01[['학원/교습소']].merge(school_district, on='학군명') ; edu_school

#### 4. 서울시 학군별 학원/교습소 학생 수 & 학교 학생 수

- 학군별 학교 학생 수

In [ ]:
# 유치원~고등학교 학생 수 합한 열 생성
school_cnt['합계'] = school_cnt['유치원']+school_cnt['초등학교']+school_cnt['중학교']+school_cnt['고등학교']

# school_cnt 데이터프레임에서 '지역', '합계'열 인덱싱하고 ' 합계'열을 기준으로 내림차순 정렬하여 sch_cnt 데이터프레임 생성
sch_cnt = school_cnt[['지역', '합계']].sort_values(by='합계', ascending=False).reset_index(drop=True)

# 열 이름 변경
sch_cnt.rename(columns = {'지역':'자치구', '합계':'학교학생수'}, inplace=True)

# sch_cnt 자치구별 해당하는 학군열 추가
col = sch_cnt['자치구']

for i in range(11):
    globals()['school_district_'+str(i+1)] = sch_cnt.query("@col in @gu_lst[@i]")


school_data_lst = [school_district_1, school_district_2, school_district_3, school_district_4, school_district_5, school_district_6, 
            school_district_7, school_district_8, school_district_9, school_district_10, school_district_11]


for k, v in zip(school_data_lst, school_district_lst):
    k.insert(0, '학군명', np.full([len(k['자치구'])], v))
    
    
sch_district = pd.concat([school_district_1, school_district_2, school_district_3, school_district_4, school_district_5, 
                             school_district_6, school_district_7, school_district_8, school_district_9, school_district_10, 
                             school_district_11])
sch_district

In [ ]:
# 학군별 학교학생수
sch_district = sch_district.groupby('학군명')[['학교학생수']].sum() ; sch_district

- 학군별 학원 학생 수 (온라인, 원격 학원 제외)

In [ ]:
# 원격, 온라인 학원 제외
학원교습소정보 = 학원교습소정보[~학원교습소정보['학원명'].str.contains('온라인')] # 온라인 in 학원명 행 제거 
학원교습소정보 = 학원교습소정보[~학원교습소정보['학원명'].str.contains('원격')] # 원격 in 학원명 행 제거
학원교습소정보 = 학원교습소정보.rename(columns = {'정원합계':'학원/교습소학생수'}) # 열 이름 변경

In [ ]:
# column = '자치구', '학원학생수' 인덱싱하여 edu_df2 데이터프레임 생성
edu_df02 = 학원교습소정보[['자치구', '학원/교습소학생수']]

# edu_df2 자치구별 해당하는 학군열 추가
col = edu_df02['자치구']

for i in range(11):
    globals()['edu_district_'+str(i+1)] = edu_df02.query("@col in @gu_lst[@i]")
    

edu_data_lst = [edu_district_1, edu_district_2, edu_district_3, edu_district_4, edu_district_5, edu_district_6, 
            edu_district_7, edu_district_8, edu_district_9, edu_district_10, edu_district_11]


for k, v in zip(edu_data_lst, school_district_lst):
    k.insert(0, '학군명', np.full([len(k['자치구'])], v))
    
    
edu_district02 = pd.concat([edu_district_1, edu_district_2, edu_district_3,  edu_district_4,  edu_district_5, 
                          edu_district_6, edu_district_7, edu_district_8, edu_district_9, edu_district_10, edu_district_11])
edu_district02

In [ ]:
# 학군별 학원학생수(온라인, 원격 제외)
edu_district02 = edu_district02.groupby('학군명')[['학원/교습소학생수']].sum() ; edu_district02

In [ ]:
# sch_deu = 학군별 학교학생수 & 학원학생수(온라인,원격 제외)
# merge / '학원학생수' 열을 기준으로 내림차순 정렬
sch_edu = sch_district[['학교학생수']].merge(edu_district02, on='학군명')
sch_edu.sort_values(by='학원/교습소학생수', ascending=False, inplace=True) ; sch_edu 

# 8학군 소재 학교에 다니는 학생보다 학원/교습소에 다니는 학생 수가 많음을 확인

### Visualize
분석 과정 또는 결과 시각화

#### 1. 서울시 자치구별 학원/교습소 수 시각화

In [ ]:
# 서울시 자치구별 학원/교습소 수 시각화
plt.figure(figsize=(12, 15))
academy.groupby('행정구역명')['학원/교습소'].count().sort_values().plot(kind='barh', color='royalblue')

plt.yticks(fontsize=12)
plt.title('서울시 자치구별 학원/교습소 수')
plt.xlabel('학원/교습소 수')
plt.show()

- 서울시 자치구별 학원/교습소 수 지도 시각화 01

In [ ]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11,
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(map)

academy_group_data = academy.groupby('행정구역명')['학원/교습소'].count().sort_values()

map.choropleth(geo_data=seoul_geo,
             data=academy_group_data, 
             fill_color='YlOrRd',
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="서울시 자치구별 학원/교습소 수"
            )

map

- 서울시 자치구별 학원/교습소 수 지도 시각화 02

In [ ]:
academy_df = pd.DataFrame(academy_group_data).reset_index()
academy_df = academy_df.rename(columns={'행정구역명':'name'})

In [ ]:
#geometry_gj = json.load(open('HangJeongDong_ver20220309.geojson', encoding='utf-8'))
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

fig = px.choropleth(academy_df, geojson=seoul_geo, locations='name', color='학원/교습소',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text='서울시 자치구별 학원/교습소 수', title_font_size=20)

#### 2. 서울시 학군별 학원/교습소 & 학교 수 시각화

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=edu_district01.index, y=edu_district01['학원/교습소'], name="학원/교습소 수"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=school_district.index, y=school_district['학교수'], name="학교 수"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="서울시 학군별 학원/교습소 & 학교 수"
)

# Set x-axis title
fig.update_xaxes(title_text="학군명")

# Set y-axes titles
fig.update_yaxes(title_text="학원/교습소 수", secondary_y=False)
fig.update_yaxes(title_text="학교 수", secondary_y=True)

fig.show()

In [ ]:
# 학군별 학원/교습소 수와 학교 수 상관관계 그래프
sns.jointplot(x='학원/교습소', y='학교수', kind='reg', height=10, data=edu_school)
# 시각화 결과 양의 상관관계를 가짐을 알 수 있음
# 학원/교습소 수 약 2,500 부터는 신뢰구간이 넓어짐

In [ ]:
# heatmap
plt.subplots(figsize=(5,5))
sns.heatmap(edu_school.corr(), annot=True, fmt='0.2f', cmap="RdYlGn", linewidths=0.2)
# 학원/교습소 수와 학교 수 상관계수 0.65

#### 3. 학군별 학원/교습소 학생 수(온라인, 원격 제외) & 학교 학생 수 시각화

In [ ]:
# 학군별 학원/교습소 학생 수 / 학교 학생 수 비율 시각화
import plotly.express as px

sch_edu['비율'] = sch_edu['학원/교습소학생수'] / sch_edu['학교학생수']

fig = px.line(sch_edu, x=sch_edu.index, y='비율', title='학군별 학원/교습소 학생 수 / 학교 학생 수 비율')
fig.show()
# 8, 9 학군이 학교 학생 수 대비 학원/교습소 학생 수가 많음
# 비율이 제일 큰 9학군은 학교 학생 수보다 학원 학생 수가 타 학군 대비 상대적으로 많음을 유추할 수 있음

In [ ]:
# 이중 Y축 선 그래프로 시각화
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=sch_edu.index, y=sch_edu['학교학생수'], name="학교 학생 수"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=sch_edu.index, y=sch_edu['학원/교습소학생수'], name="학원/교습소 학생 수"),
    secondary_y=True,
)

fig.update_layout(
    title_text="서울시 학군별 학원/교습소 & 학교 학생 수"
)

fig.update_xaxes(title_text="학군명")

fig.update_yaxes(title_text="학원/교습소 학생 수", secondary_y=False)
fig.update_yaxes(title_text="학교 학생 수", secondary_y=True)

fig.show()

# 8, 9학군이 타 학군에 비해 학원/교습소 학생 수가 압도적으로 많음 > 타 학군 소재 학교 학생들이 8, 9 학군 소재 학원에 등원함을 유추
# 위 비율 시각화에서 유추한대로 9학군은 학교 학생 수보다 학원/교습소 학생 수가 타 학군 대비 상대적으로 많음을 볼 수 있음

In [ ]:
# 학군별 학원/교습소 힉생 수와 학교 학생 수 상관관계 그래프
sns.jointplot(x='학교학생수', y='학원/교습소학생수', kind='reg', height=10, data=sch_edu)
# 양의 상관관계를 보이나 전반적으로 신뢰구간이 넓음

In [ ]:
# 학교 학생 수와 학원/교습소 학생 수의 상관계수 0.33
plt.subplots(figsize=(5,5))
sns.heatmap(sch_edu[['학교학생수', '학원/교습소학생수']].corr(), annot=True, fmt='0.2f', cmap="RdYlGn", linewidths=0.2)